# Information-Flow Clustering for 6-task experiments

In [ ]:
run_id = "IFC-6task-000"

dataset="MPC"
# dataset="TIC"
# dataset="KEF"


# arch="vgg16"
# lr,lrmin,patience,lrfactor = 0.01, 0.001, 5, 3
# sparsity,share_sparsity_offset, numfilters, bs = 0.65, 0.05, 64, 64

arch="modresnet18"
lr,lrmin,patience,lrfactor = 0.1, 0.001, 5, 5
sparsity,share_sparsity_offset, numfilters, bs = 0.7, 0.05, 64, 64

### "structured" performs standard IFC, while "unstructured" does IFC-US
# prune_method="structured"
prune_method="unstructured"

tr_epochs, ft_epochs, shared_layers, score_threshold, num_cluster_layers = 40, 30, -1, 2.0, 4

### Note: Number of subnetworks shared doesnt affect clustering or optimal manual sharing, and "1" is used as a placeholder value
sim_type, num_shared, sharetype, shareorder = "corr", 1, "clustering", "lowest"
# sim_type, num_shared, sharetype, shareorder = "corr", 1, "optimalmanual", "lowest"
# sim_type, num_shared, sharetype, shareorder = "corr", 6, "standard", "lowest"
# sim_type, num_shared, sharetype, shareorder = "corr", 0, "standard", "lowest"


!python "main.py" --arch=$arch --dataset=$dataset --run_id=$run_id \
--similarity_type=$sim_type --num_shared=$num_shared --shareorder=$shareorder --share_type=$sharetype \
--prune_perc_per_layer=$sparsity --batch_size=$bs --train_epochs=$tr_epochs --finetune_epochs=$ft_epochs --task_order 0 1 2 3 4 5 --num_tasks=6 \
--task_num=0 --num_filters=$numfilters --lr=$lr --lr_patience=$patience --lr_factor=$lrfactor --lr_min=$lrmin \
--num_cluster_layers=$num_cluster_layers --score_threshold=$score_threshold --share_sparsity_offset=$share_sparsity_offset \
--shared_layers=$shared_layers --prune_method=$prune_method



!python "Eval T Tasks.py" --arch=$arch --dataset=$dataset --run_id=$run_id \
--similarity_type=$sim_type --num_shared=$num_shared --shareorder=$shareorder --share_type=$sharetype \
--prune_perc_per_layer=$sparsity --batch_size=$bs --train_epochs=$tr_epochs --finetune_epochs=$ft_epochs --task_order 0 1 2 3 4 5 --num_tasks=6 \
--task_num=6 --num_filters=$numfilters --lr=$lr --lr_patience=$patience --lr_factor=$lrfactor --lr_min=$lrmin \
--num_cluster_layers=$num_cluster_layers --score_threshold=$score_threshold --share_sparsity_offset=$share_sparsity_offset \
--shared_layers=$shared_layers --prune_method=$prune_method






# Two-Task Subnetwork Usefulness Experiments

In [ ]:
run_id = "IFC-2task-000"


dataset = "MPC"
# dataset = "TIC"
# dataset = "KEF"



arch="vgg16"
lr,lrmin,patience,lrfactor = 0.01, 0.001, 5, 3
sparsity,share_sparsity_offset, numfilters, bs = 0.65, 0.05, 64, 64

# arch="modresnet18"
# lr,lrmin,patience,lrfactor = 0.1, 0.001, 5, 5
# sparsity,share_sparsity_offset, numfilters, bs = 0.7, 0.05, 64, 64


### Two-task experiments always train on a pair of tasks, and always share the first task's subnetwork
num_tasks = 2
num_shared = 1
train_epochs, finetune_epochs, shared_layers, score_threshold, num_cluster_layers = 3, 3, -1, 2.0, 4

# prune_method="structured"
prune_method="unstructured"





### Note: Number of subnetworks shared doesnt affect clustering or optimal manual sharing, and "1" is used as a placeholder value
sim_type, num_shared, sharetype, shareorder = "corr", 1, "standard", "lowest"


### When task_num is 1, only task B is trained using the checkpoint from task A
### The first time a given task A is trained, task_num must be 0, subsequent sequences we set task_num to 1
for a in range(0,6):
  task_num = 0
  for b in range(0,6):
    if b > 0:
      task_num = 1


    ### Runs the training for this pair of tasks a and b
    !python "main.py" --arch=$arch --dataset=$dataset --run_id=$run_id \
    --similarity_type=$sim_type --num_shared=$num_shared --shareorder=$shareorder --share_type=$sharetype \
    --prune_perc_per_layer=$sparsity --batch_size=$bs --train_epochs=$train_epochs --finetune_epochs=$finetune_epochs --task_order $a $b --num_tasks=2 \
    --task_num=$task_num --num_filters=$numfilters --lr=$lr --lr_patience=$patience --lr_factor=$lrfactor --lr_min=$lrmin \
    --num_cluster_layers=$num_cluster_layers --score_threshold=$score_threshold --share_sparsity_offset=$share_sparsity_offset \
    --shared_layers=$shared_layers --prune_method=$prune_method
